In [1]:
import copy
def returnBoard(filename, rows, columns):
    # SRC: http://stackoverflow.com/questions/3277503/how-to-read-a-file-line-by-line-into-a-list
    with open(filename) as f:
        content = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
        content = [x.split() for x in content] 
    board = []
    for row in range(rows):
        currentRow = []
        for column in range(columns):
            if((content[row][column])=='-'):
                value = 0
            else:
                value = int(content[row][column])
            currentRow.append(value)
        board.append(currentRow)
        # print currentRow
    return board
    
    
class SudokuPuzzle:
    def __init__(self, possibleValues, filename, board=None, initialize=True, rows=9, columns=9, boxRows=3, boxColumns=3):
        if(initialize):
            self.variableSize = rows * columns
            self.possibleValues = possibleValues
            self.rows = rows
            self.columns = columns
            self.boxRows = boxRows
            self.boxColumns = boxColumns
            if(board==None):
                self.board = returnBoard(filename,rows,columns)
            else:
                self.board = board
            # print "Creating domainVariables"
            self.updateDomainVariables()
            # print self.domainVariables
            
    def rowAllDiff(self, row):
        dictV = {}
        for column in range(self.columns):
            if(self.board[row][column] in dictV):
                # print "Row ALL DIFF Failed"
                return False
            else:
                if(self.board[row][column]!=0):
                    dictV[self.board[row][column]] = 1
        return True
    
    def columnAllDiff(self, column):
        dictV = {}
        for row in range(self.rows):
            if(self.board[row][column] in dictV):
                # print "Col ALL DIFF Failed"
                return False
            else:
                if(self.board[row][column]!=0):
                    dictV[self.board[row][column]] = 1
        return True
    
    def boxAllDiff(self, r, c):
        
        dictV = {}
        boxRowStart = self.boxRows*(r/self.boxRows)
        boxColumnStart = self.boxColumns*(c/self.boxColumns)
        boxRowEnd = boxRowStart + self.boxRows
        boxColumnEnd = boxColumnStart + self.boxColumns
        # print "CHECKING BOX (%d, %d) " %(r,c) 
        for row in range(boxRowStart, boxRowEnd):
            for column in range(boxColumnStart, boxColumnEnd):
                # print (row,column)
                if(self.board[row][column] in dictV):
                    # self.printBoard()
                    # print "Board ALL DIFF Failed"
                    return False
                else:
                    if(self.board[row][column]!=0):
                        dictV[self.board[row][column]] = 1
        return True
    
    def checkConstraints(self, row, column):
        toCheckVal = self.board[row][column]
        return self.rowAllDiff(row) and self.columnAllDiff(column) and self.boxAllDiff(row, column)
    
    def reachedGoal(self):
        for row in range(self.rows):
            for column in range(self.columns):
                if(self.board[row][column]==0):
                    return False
        for row in range(self.rows):
            for column in range(self.columns):
                if(self.checkConstraints(row,column)):
                    continue
                else:
                    return None
        return True
    
    def assignment(self, row, column, value):
        self.board[row][column] = value
        self.updateDomainVariables()
    
    def backtrackAssignment(self, row, column, value):
        self.board[row][column] = value
        
    def updateDomainVariables(self):
        domain = {}
        for value in self.possibleValues:
            domain[value] = 1;
        boxAvailableValues = []
        rowAvailableValues = []
        columnAvailableValues = []
        # copy.copy(domain)
        for row in range(self.rows):
            rowAvailableValues.append(copy.deepcopy(domain))
        for column in range(self.columns):
            columnAvailableValues.append(copy.deepcopy(domain))
        
        totalBoxes = (self.rows * self.columns)/(self.boxRows*self.boxColumns)
        for box in range(totalBoxes):
            boxAvailableValues.append(copy.deepcopy(domain))
            
        variableDomain = []
        # Add used values 
        for row in range(self.rows):
            variableDomain.append([])
            for column in range(self.columns):
                if(self.board[row][column]!=0):
                    boxAvailableValues[((3*(row/3))+(column/3))][self.board[row][column]] = 0
                    rowAvailableValues[row][self.board[row][column]] = 0
                    columnAvailableValues[column][self.board[row][column]] = 0

        for row in range(self.columns):
            for column in range(self.columns):  
                if(self.board[row][column]==0):
                    variableDomain[row].append(self.findPositionsPossibleValues(row, column, boxAvailableValues, rowAvailableValues, columnAvailableValues))
                else:
                    variableDomain[row].append(self.board[row][column])
        # print variableDomain, boxAvailableValues, rowAvailableValues, columnAvailableValues
        self.variableValuesAvailable = variableDomain
        self.boxValuesAvailable = boxAvailableValues
        self.rowValuesAvailable = rowAvailableValues
        self.columnValuesAvailable = columnAvailableValues
        return variableDomain, boxAvailableValues, rowAvailableValues, columnAvailableValues
        
    def findPositionsPossibleValues(self, row, column, boxAvailableValues, rowAvailableValues, columnAvailableValues):
        BoxIndex = ((3*(row/3))+(column/3))
        boxHash = boxAvailableValues[BoxIndex]
        boxValueSet = set({k: v for k, v in boxHash.iteritems() if boxHash[k]==1})
        rowHash = rowAvailableValues[row]
        rowValueSet = set({k: v for k, v in rowHash.iteritems() if rowHash[k]==1})
        columnHash = columnAvailableValues[column]
        columnValueSet = set({k: v for k, v in columnHash.iteritems() if columnHash[k]==1})
        return boxValueSet & columnValueSet & rowValueSet

        
    def printBoard(self):
        for row in range(self.rows):
            print self.board[row]
            
    def printAvailableValues(self):
        for row in range(self.rows):
            for column in range(self.columns):
                if(self.board[row][column]==0):
                    print "Values available for row %d column %d are " %(row, column) 
                    print self.variableValuesAvailable[row][column]

    def reachedFailure(self):
        for row in range(self.rows):
            for column in range(self.columns):
                if( (self.board[row][column]==0) and len(self.variableValuesAvailable[row][column])==0):
                    return True
                if(self.checkConstraints(row,column)):
                    continue
                else:
                    return False
        return False
    
    def updateBoardAndDomainValues(self):
        for row in range(self.rows):
            for column in range(self.columns):
                if((self.board[row][column]==0) and len(self.variableValuesAvailable[row][column])==1):
                    #print "Adding the only elt to the board for %d, %d" %(row,column)
                    self.board[row][column] = list(self.variableValuesAvailable[row][column])[0]
        self.updateDomainVariables()
        #print "Updated Domain Variables"
        #print self.variableValuesAvailable
        needToCallAgain = False
        for row in range(self.rows):
            for column in range(self.columns):
                if((self.board[row][column]==0) and len(self.variableValuesAvailable[row][column])==1):
                    needToCallAgain = True
                    break
        if(needToCallAgain):
            self.updateBoardAndDomainValues()
        return
        
                    
    def revise(self, Xi, Xj):
        revised = False
        # if(Xi[0] == 1 and Xi[1] == 2):
            # pdb.set_trace()
        row, column = Xi
        otherRow, otherColumn = Xj
        if((self.board[row][column])!=0):
            varValuesAvailbleForXi = [self.board[row][column]]
        else:
            varValuesAvailbleForXi = copy.deepcopy(self.variableValuesAvailable[row][column])

        if((self.board[otherRow][otherColumn])!=0):
            varValuesAvailbleForXj = [self.board[otherRow][otherColumn]]
        else:
            varValuesAvailbleForXj = copy.deepcopy(self.variableValuesAvailable[otherRow][otherColumn]) 
    
        # print "---------------------------"
        # print "Revise Function Xi : (%d, %d), Xj : (%d, %d)" %(Xi[0],Xi[1],Xj[0],Xj[1])
        # print "varValuesAvailbleForXi : "
        # print varValuesAvailbleForXi
        # print "varValuesAvailbleForXj :"
        # print varValuesAvailbleForXj
        for valueAvailableForXi in varValuesAvailbleForXi:
            satisfiedValue = False
            
            for otherValueAvailableForXj in varValuesAvailbleForXj:
                if(otherValueAvailableForXj!=valueAvailableForXi):
                    satisfiedValue = True
            if(satisfiedValue == False):
                self.variableValuesAvailable[row][column].remove(valueAvailableForXi)
                revised = True
        # print "Revised Value " + str(revised)
        # print "---------------------------"
        return revised
        
                    
    def addAllConstraintsAvailable(self):
        Constraints = []
        boxRowStart = 0 
        for boxRow in range(self.boxRows):
            boxColumnStart = 0
            for boxColumn in range(self.boxColumns):
                for value in self.boxValueCombinations(boxRowStart, boxColumnStart):
                    Constraints.append(value)
                boxColumnStart+=self.boxColumns
            boxRowStart += self.boxRows
        
        for row in range(self.rows):
            ColumnVariables = []
            for column in range(self.columns):
                ColumnVariables.append((row,column))
            for value in itertools.combinations(ColumnVariables, 2):
                Constraints.append(value)
        
        for column in range(self.columns):
            RowVariables = []
            for row in range(self.rows):
                RowVariables.append((row,column))
            for value in itertools.combinations(RowVariables, 2):
                Constraints.append(value)
        
        
        return Constraints #itertools.combinations(Variables, 2)
    
    def boxValueCombinations(self, r, c):
        boxVariables = []
        boxRowStart = self.boxRows*(r/self.boxRows)
        boxColumnStart = self.boxColumns*(c/self.boxColumns)
        boxRowEnd = boxRowStart + self.boxRows
        boxColumnEnd = boxColumnStart + self.boxColumns
        for row in range(boxRowStart, boxRowEnd):
            for column in range(boxColumnStart, boxColumnEnd):
                boxVariables.append((row, column))
        return itertools.combinations(boxVariables, 2)

    def findNeighbours(self, rcInd):
        r,c = rcInd
        Neighbours = set()
        for row in range(self.rows):
            Neighbours.add((row, c))
        for column in range(self.columns):
            Neighbours.add((r, column))
        
        boxRowStart = self.boxRows*(r/self.boxRows)
        boxColumnStart = self.boxColumns*(c/self.boxColumns)
        boxRowEnd = boxRowStart + self.boxRows
        boxColumnEnd = boxColumnStart + self.boxColumns
        for row in range(boxRowStart, boxRowEnd):
            for column in range(boxColumnStart, boxColumnEnd):
                Neighbours.add((row, column))

        Neighbours.remove((r,c))
#         print "Neighbours of (%d %d) ARE" %(r,c)
#         print Neighbours
        return list(Neighbours)


In [2]:
count=0
searchcount=0
def mrv(puzzle):
    minval=100
    for row in range(puzzle.rows):
        for column in range(puzzle.columns):
            if((puzzle.board[row][column]==0) and len(puzzle.variableValuesAvailable[row][column])<minval ):
                minval=len(puzzle.variableValuesAvailable[row][column])
                rowvalue=row
                columnvalue=column
    
    return rowvalue,columnvalue
def BACKTRACKINGSEARCH(puzzle): 
    return BACKTRACK(puzzle)

def BACKTRACK(puzzle, simple_backtracking=False):
    global count
    global searchcount
   
    # print "**********"
    # puzzle.printBoard()
    # print "**********"
#     if(!simple_backtracking):
        # DOES FORWARD CHECKING, AN INFERENCE METHOD
    puzzle.updateBoardAndDomainValues()
    reached = puzzle.reachedGoal()
    if(reached == None):
        return False
    if(reached):
        print "SOLUTION REACHED"
        puzzle.printBoard()
        # print puzzle.columnAllDiff(1);
        return puzzle
    if(puzzle.reachedFailure()):
        # print "Failure Reached"
        # puzzle.printBoard()
        
        return False
    
    print count,searchcount
    rowvalues,columnvalues=mrv(puzzle)
    for row in [rowvalues]:
        for column in [columnvalues]:
            if(puzzle.board[row][column]==0):
                
                # print "Reached %d %d Pos" %(row, column)
                # orderedBestPossibleValues = OrderedDomainValues(puzzle, row, column)
                # puzzle.printAvailableValues()
                # if(len(puzzle.variableValuesAvailable[row][column])==0):
                #    return False
                ValueFromRowColumnSatisfied = False
                possibleValuesInDomain = puzzle.possibleValues
                
                count+=1
                for value in puzzle.variableValuesAvailable[row][column]:
                    searchcount+=1
                    board = copy.deepcopy(puzzle.board)
                    new_puzzle = SudokuPuzzle(possibleValues, filename, board)
                    new_puzzle.assignment(row, column, value)
                    #print "Assigning position (%d %d) a value of %d" %(row, column, value)
                    if(not new_puzzle.checkConstraints(row,column)):
                        # print "CONSTRAINTS FAILED For value : %d" %(value)
                        # new_puzzle.printBoard()
                        continue # assignment was a bad assignment
                    # print "value : %d" %(value)
                    
                    # new_puzzle.printBoard()
                    result = BACKTRACK(new_puzzle)
                    # ValueFromRowColumnSatisfied = result or ValueFromRowColumnSatisfied
                    if(result == False):
                        continue
                    else:
                        ValueFromRowColumnSatisfied = True
                        return result
                if(not ValueFromRowColumnSatisfied):
                    #print "BackTracking for ROW %d COLUMN %d" %(row, column)
                    return False


In [3]:
def SIMPLEBACKTRACK(puzzle, simple_backtracking=False):
#     print "**********"
#     puzzle.printBoard()
#     print "**********"
#     if(!simple_backtracking):
        # DOES FORWARD CHECKING, AN INFERENCE METHOD
#     puzzle.updateBoardAndDomainValues()
    reached = puzzle.reachedGoal()
    if(reached == None):
        return False
    if(reached):
        print "SOLUTION REACHED"
        puzzle.printBoard()
        # print puzzle.columnAllDiff(1);
        return puzzle
    if(puzzle.reachedFailure()):
        # print "Failure Reached"
        # puzzle.printBoard()
        
        return False
    
    global count
    global searchcount
   
    for row in range(puzzle.rows):
        for column in range(puzzle.columns):
            if(puzzle.board[row][column]==0):
                count+=1;
                #print "Reached %d %d Pos" %(row, column)
                # orderedBestPossibleValues = OrderedDomainValues(puzzle, row, column)
                # puzzle.printAvailableValues()
                # if(len(puzzle.variableValuesAvailable[row][column])==0):
                #    return False
                ValueFromRowColumnSatisfied = False
                possibleValuesInDomain = puzzle.possibleValues

                for value in possibleValuesInDomain:
                    searchcount+=1
                    board = copy.deepcopy(puzzle.board)
                    new_puzzle = SudokuPuzzle(possibleValues, filename, board)
                    new_puzzle.backtrackAssignment(row, column, value)
                    #print "Assigning position (%d %d) a value of %d" %(row, column, value)
                    if(not new_puzzle.checkConstraints(row,column)):
                        # print "CONSTRAINTS FAILED For value : %d" %(value)
#                         print "**********"
#                         new_puzzle.printBoard()
#                         print "**********"
                        continue # assignment was a bad assignment
                    # print "value : %d" %(value)
                    
                    # new_puzzle.printBoard()
                    result = SIMPLEBACKTRACK(new_puzzle)
                    # ValueFromRowColumnSatisfied = result or ValueFromRowColumnSatisfied
                    if(result == False):
                        continue
                    else:
                        ValueFromRowColumnSatisfied = True
                        return result
                if(not ValueFromRowColumnSatisfied):
                    #print "BackTracking for ROW %d COLUMN %d" %(row, column)
                    return False


In [13]:
possibleValues = [1,2,3,4,5,6,7,8,9]
filename = "sudoku_sample3.txt"
filename = "puzzle/puz-081.txt"
def solveSudoku():
    global count
    global searchcount
    count=0
    searchcount=0
    puzzle = SudokuPuzzle(possibleValues, filename)
    puzzle.printBoard()
    # puzzle.printAvailableValues()
#     return puzzle
    
    return BACKTRACKINGSEARCH(puzzle)

puzzle = solveSudoku()
print searchcount-count
#puzzle.printBoard()

[0, 2, 0, 0, 3, 0, 0, 0, 4]
[0, 1, 9, 0, 2, 0, 0, 8, 0]
[7, 0, 8, 0, 0, 0, 0, 0, 0]
[2, 7, 6, 0, 0, 4, 3, 0, 0]
[8, 0, 0, 0, 0, 0, 0, 0, 6]
[0, 0, 0, 0, 8, 0, 0, 0, 0]
[0, 0, 0, 0, 6, 0, 0, 9, 5]
[0, 0, 0, 5, 0, 9, 0, 0, 0]
[0, 0, 3, 7, 0, 0, 0, 0, 0]
0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 14
12 17
13 18
14 19
15 20
16 21
17 22
18 26
19 29
20 30
21 31
22 32
23 33
24 36
25 37
26 38
27 39
28 40
29 41
30 45
31 48
32 49
33 50
34 51
35 52
36 53
37 57
38 60
39 61
40 64
41 65
SOLUTION REACHED
[6, 2, 5, 8, 3, 7, 9, 1, 4]
[3, 1, 9, 4, 2, 6, 5, 8, 7]
[7, 4, 8, 9, 5, 1, 6, 3, 2]
[2, 7, 6, 1, 9, 4, 3, 5, 8]
[8, 9, 4, 3, 7, 5, 1, 2, 6]
[5, 3, 1, 6, 8, 2, 7, 4, 9]
[1, 8, 7, 2, 6, 3, 4, 9, 5]
[4, 6, 2, 5, 1, 9, 8, 7, 3]
[9, 5, 3, 7, 4, 8, 2, 6, 1]
26


In [5]:
from collections import deque
import itertools

def AC_3(puzzle, assignedIndex):
    # print "AC3 Algorithm ----"
    row, column = assignedIndex
    result = puzzle.addAllConstraintsAvailable()
#     result = puzzle.findNeighbours(assignedIndex)
#     if(result == None):
#         return True
    Deque = deque()
    for value in result:
        Deque.append(value)
    #print Deque
    while(len(Deque)>0):
        Xi, Xj = Deque[0]
        # print Xi,Xj
        Deque.popleft()
        # print "Puzzle is being Checked Xi: "+ str(Xi) + " Xj : " + str(Xj)
        if(puzzle.revise(Xi,Xj)):
            # print "Puzzle is being Revised Xi: "+ str(Xi) + " Xj : " + str(Xj)
            if(puzzle.board[Xi[0]][Xi[1]] ==0 and len(puzzle.variableValuesAvailable[Xi[0]][Xi[1]])==0):
                # print "AC 3 Inference Failure Failed Because of Conflicts"
                return False
            Xi_neighbours = puzzle.findNeighbours(Xi)
            # print "Neighbours of (%d %d) ARE" %(Xi[0],Xi[1])
            # print Xi_neighbours
            if(len(Xi_neighbours)>0):
                for value in (Xi_neighbours):
                    if(value != Xj):
                        Deque.append((value, Xi))

    return True

count=0
searchcount=0

def SolveUsingAC3(puzzle, simple_backtracking=False):
    global count
    global searchcount
   
    # print "**********"
    # puzzle.printBoard()
    # print "**********"
#     if(!simple_backtracking):
        # DOES FORWARD CHECKING, AN INFERENCE METHOD
    #puzzle.updateBoardAndDomainValues()
    reached = puzzle.reachedGoal()
    if(reached == None):
        return False
    if(reached):
        print "SOLUTION REACHED"
        puzzle.printBoard()
        # print puzzle.columnAllDiff(1);
        return puzzle
    if(puzzle.reachedFailure()):
        # print "Failure Reached"
        # puzzle.printBoard()
        
        return False
    
    # print count,searchcount
    rowvalues,columnvalues=mrv(puzzle)
    for row in [rowvalues]:
        for column in [columnvalues]:
            if(puzzle.board[row][column]==0):
                
                # print "Reached %d %d Pos" %(row, column)
                # orderedBestPossibleValues = OrderedDomainValues(puzzle, row, column)
                # puzzle.printAvailableValues()
                # if(len(puzzle.variableValuesAvailable[row][column])==0):
                #    return False
                ValueFromRowColumnSatisfied = False
                
                count+=1
                for value in puzzle.variableValuesAvailable[row][column]:
                    searchcount+=1
                    board = copy.deepcopy(puzzle.board)
                    new_puzzle = SudokuPuzzle(possibleValues, filename, board)
                    new_puzzle.assignment(row, column, value)
                    # print "Assigning position (%d %d) a value of %d" %(row, column, value)
                    if(not AC_3(new_puzzle, (row, column))):
                        # print "CONSTRAINTS FAILED For value : %d" %(value)
                        # new_puzzle.printBoard()
                        continue # assignment was a bad assignment
                    # print "value : %d" %(value)
                    
                    # new_puzzle.printBoard()
                    result = SolveUsingAC3(new_puzzle)
                    if(result == False):
                        continue
                    else:
                        ValueFromRowColumnSatisfied = True
                        return result
                if(not ValueFromRowColumnSatisfied):
                    #print "BackTracking for ROW %d COLUMN %d" %(row, column)
                    return False


In [14]:
possibleValues = [1,2,3,4,5,6,7,8,9]
filename = "sudoku_sample_4.txt"
filename = "puzzle/puz-081.txt"
def solveSudokuUsingAC3():
    global count
    global searchcount
    count=0
    searchcount=0
    puzzle = SudokuPuzzle(possibleValues, filename)
    puzzle.printBoard()
    return SolveUsingAC3(puzzle)

puzzle = solveSudokuUsingAC3()
print searchcount-count


[0, 2, 0, 0, 3, 0, 0, 0, 4]
[0, 1, 9, 0, 2, 0, 0, 8, 0]
[7, 0, 8, 0, 0, 0, 0, 0, 0]
[2, 7, 6, 0, 0, 4, 3, 0, 0]
[8, 0, 0, 0, 0, 0, 0, 0, 6]
[0, 0, 0, 0, 8, 0, 0, 0, 0]
[0, 0, 0, 0, 6, 0, 0, 9, 5]
[0, 0, 0, 5, 0, 9, 0, 0, 0]
[0, 0, 3, 7, 0, 0, 0, 0, 0]
SOLUTION REACHED
[6, 2, 5, 8, 3, 7, 9, 1, 4]
[3, 1, 9, 4, 2, 6, 5, 8, 7]
[7, 4, 8, 9, 5, 1, 6, 3, 2]
[2, 7, 6, 1, 9, 4, 3, 5, 8]
[8, 9, 4, 3, 7, 5, 1, 2, 6]
[5, 3, 1, 6, 8, 2, 7, 4, 9]
[1, 8, 7, 2, 6, 3, 4, 9, 5]
[4, 6, 2, 5, 1, 9, 8, 7, 3]
[9, 5, 3, 7, 4, 8, 2, 6, 1]
7
